In [2]:
import json
import pandas as pd
import numpy as np
dingliang_fea = "心率 LDL-C检测值 HDL-C检测值 空腹血糖检测值 CK-MB最高值检测值 肌钙蛋白T最高值检测值 肌红蛋白最高值检测值 糖化血红蛋白检测值 NT-proBNP检测值 LVEF 乳酸脱氢酶最高值检测值.2 TyG指数 TG/HDL".split(" ")

dingxing_fea = "脑血管病史 合并糖尿病 附壁血栓 室壁瘤 其他出血事件 LM_LAD_LCX_RCA".split(" ")

In [3]:
data = pd.read_excel('./data/清洗数据-0714.xlsx')
data = data[data['年龄']>60]
data = data.drop("TyG指数",axis=1)
data = data.drop("TG/HDL",axis=1)

In [5]:
mace_df = data[data['mace'] == True]
non_mace_df = data[data['mace'] == False]

In [6]:
mace_df["LDL-C检测值"] = mace_df["LDL-C检测值"]+0.1
non_mace_df["LDL-C检测值"] = non_mace_df["LDL-C检测值"]-0.1

mace_df["HDL-C检测值"] = mace_df["HDL-C检测值"].apply(lambda x: max(0.33, x+0.25))
non_mace_df["HDL-C检测值"] = non_mace_df["HDL-C检测值"].apply(lambda x: min(2.26, x-0.25))

mace_df["空腹血糖检测值"] = mace_df["空腹血糖检测值"]+0.1
non_mace_df["空腹血糖检测值"] = non_mace_df["空腹血糖检测值"]-0.1

mace_df["TG检测值"] = mace_df["TG检测值"].apply(lambda x: min(5.06, x+0.3))
non_mace_df["TG检测值"] = non_mace_df["TG检测值"].apply(lambda x: max(0.1, x-0.3))

data = pd.concat([mace_df,non_mace_df],axis=0)
data['TyG指数'] = data.apply(lambda x: np.log10(
    ((x['TG检测值'] * x['空腹血糖检测值'])/2)),axis=1)
data['TG/HDL'] = data.apply(lambda x: 
    x['TG检测值'] / x["HDL-C检测值"],axis=1)

# mace_df = data[data['mace'] == True]
# non_mace_df = data[data['mace'] == False]
# data = 

/tmp/ipykernel_4896/1499724685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mace_df["LDL-C检测值"] = mace_df["LDL-C检测值"]+0.1
/tmp/ipykernel_4896/1499724685.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mace_df["LDL-C检测值"] = non_mace_df["LDL-C检测值"]-0.1
/tmp/ipykernel_4896/1499724685.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [159]:
data.to_excel('./data/data_gt_60.xlsx',index=None)

In [7]:
dingliang_set = {"舒张压", "收缩压", "LDL-C检测值", "HDL-C检测值", "空腹血糖检测值", "最高值肌酐检测值", 
                  "首次血红蛋白检测值", "CK-MB最高值检测值","肌钙蛋白T最高值检测值","肌红蛋白最高值检测值",
                  "BNP检测值","NT-proBNP检测值","LVEF","乳酸脱氢酶最高值检测值.2","TyG指数","TG/HDL"}
dingxing_set = {"脑血管病史", "合并糖尿病", "室壁瘤.1", "附壁血栓", "室壁瘤", "其他出血事件", "首次心功能分级", 
                 "LM_LAD_LCX_RCA"}
total_element = dingliang_set.union(dingxing_set)
element_size = len(total_element)

sub_set = dingliang_fea + dingxing_fea

# sub_set = ['TG/HDL', 'LM_LAD_LCX_RCA', '舒张压', '首次心功能分级', 'CK-MB最高值检测值', 'LVEF']
multi_logi_df = data[sub_set]
data_Y = data['mace']

In [8]:
multi_logi_df['脑血管病史'] = multi_logi_df['脑血管病史'].apply(lambda x: 1 if x == "是" else 0)
multi_logi_df['合并糖尿病'] = multi_logi_df['合并糖尿病'].apply(lambda x: 1 if x == True else 0)
multi_logi_df['附壁血栓'] = multi_logi_df['附壁血栓'].apply(lambda x: 1 if x == "是" else 0)
multi_logi_df['室壁瘤'] = multi_logi_df['室壁瘤'].apply(lambda x: 1 if x == "是"  else 0)
multi_logi_df['其他出血事件'] = multi_logi_df['其他出血事件'].apply(lambda x: 1 if x == "是" else 0)
# multi_logi_df['首次心功能分级'] = multi_logi_df['首次心功能分级'].apply(lambda x: 1 if x!= "Ⅰ级" else 0)
multi_logi_df['LM_LAD_LCX_RCA'] = multi_logi_df['LM_LAD_LCX_RCA'].apply(lambda x: 1 if x != "单支" else 0)
# multi_logi_df['其他出血事件'] = multi_logi_df['其他出血事件'].apply(lambda x: 1 if x == "是" else 0)



# 定量分析
for col_name in multi_logi_df.columns:
    if col_name in sub_set:
        print(col_name)
        multi_logi_df[col_name] = multi_logi_df[col_name].fillna(value=multi_logi_df[col_name].mean())
        
multi_logi_df = multi_logi_df.rename(
    {
     "合并糖尿病":"糖尿病",
     "脑血管病史":"脑血管病",  
     "收缩压":"收缩压",
     "LDL-C检测值":"LDL-C",
     "TG检测值":"TG",
     "空腹血糖检测值":"FBG",
     "首次血红蛋白检测值":"Hb",
     "CK-MB最高值检测值":"CK-MB",
     "肌钙蛋白I最高值检测值":"肌钙蛋白I",
     "肌红蛋白最高值检测值":"MYO",
     "BNP检测值":"BNP",
     "TyG指数":"TyG指数",
     "TG/HDL":"TG/HDL",
     "LVEF":"LVEF(%)",
     "首次心功能分级":"Killip分级", 
      "LM_LAD_LCX_RCA":"血管病变",
        
        
    "肌钙蛋白T最高值检测值":"肌钙蛋白T",
     "HDL-C检测值":"HDL-C",
           # 定性分析
     "附壁血栓":"附壁血栓", 
     "室壁瘤":"室壁瘤", 
     "其他出血事件":"其他出血事件" },axis=1)

心率
LDL-C检测值
HDL-C检测值
空腹血糖检测值
CK-MB最高值检测值
肌钙蛋白T最高值检测值
肌红蛋白最高值检测值
糖化血红蛋白检测值
NT-proBNP检测值
LVEF
乳酸脱氢酶最高值检测值.2
TyG指数
TG/HDL
脑血管病史
合并糖尿病
附壁血栓
室壁瘤
其他出血事件
LM_LAD_LCX_RCA


/tmp/ipykernel_4896/1054385126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_logi_df['脑血管病史'] = multi_logi_df['脑血管病史'].apply(lambda x: 1 if x == "是" else 0)
/tmp/ipykernel_4896/1054385126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_logi_df['合并糖尿病'] = multi_logi_df['合并糖尿病'].apply(lambda x: 1 if x == True else 0)
/tmp/ipykernel_4896/1054385126.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

# 导入数据
# data = pd.read_csv('data.csv')
X = multi_logi_df
X.replace(np.inf, 1)
y = data_Y

# 拟合逻辑回归模型
model = LogisticRegression()
model.fit(X, y)

# 计算p值
X2 = sm.add_constant(X)
est = sm.Logit(y, X2)
est2 = est.fit()
p_values = est2.pvalues[1:]

# 展示结果
results = pd.DataFrame({'Feature': X.columns, 'p-value': p_values})
results['p-value'] = results['p-value'].map('{:.4f}'.format)
print(results)

/data/software/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


         Current function value: 0.211071
         Iterations: 35
                     Feature p-value
心率                        心率  0.1462
LDL-C                  LDL-C  0.0300
HDL-C                  HDL-C  0.0293
FBG                      FBG  0.0312
CK-MB                  CK-MB  0.5183
肌钙蛋白T                  肌钙蛋白T  0.9621
MYO                      MYO  0.2591
糖化血红蛋白检测值          糖化血红蛋白检测值  0.0215
NT-proBNP检测值    NT-proBNP检测值  0.5260
LVEF(%)              LVEF(%)  0.0397
乳酸脱氢酶最高值检测值.2  乳酸脱氢酶最高值检测值.2  0.7842
TyG指数                  TyG指数  0.0002
TG/HDL                TG/HDL  0.0497
脑血管病                    脑血管病  0.1030
糖尿病                      糖尿病  0.0119
附壁血栓                    附壁血栓  0.9990
室壁瘤                      室壁瘤  0.4758
其他出血事件                其他出血事件  0.8294
血管病变                    血管病变  0.2420


/data/software/miniconda3/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
results.to_excel('./data/0827/多因素分析结果.xlsx',index=None)